# Fast Poisson Equation Solver

## Introduction

todo

## Our approach
- Find better vertex orderings of the Poisson matrix so that the Cholesky factorization can be solved fast

## Hypothesis
- We start with simple, greedy orderings. For example, we order vertice by their degrees, by their x-coordinates, or simply randomly.

We first start with a toy implementation of the column-oriented Cholesky factorization:

In [1]:
import numpy as np
from math import sqrt

def cholesky(m):
    rows, cols = m.shape[0], m.shape[1]
    if (rows == 1):
        m[(0, 0)] = sqrt(m[(0, 0)])
        return m
    m[(0, 0)] = sqrt(m[(0, 0)])
    r = np.vectorize(lambda x: x/m[(0, 0)])(m[0, 1:])
    for i in xrange(1, rows):
        m[(0, i)] /= m[(0, 0)]
        m[(i, 0)] /= m[(0, 0)]
    to_subtract = r.T*r
    sub = m[1:rows, 1:cols]
    sub -= to_subtract
    m[np.ix_([1, rows-1], [1, cols-1])] = cholesky(sub)
    return m

# test input
m = np.matrix([[4, 12, -16], [12, 37, -43], [-16, -43, 98]])
print "* original matrix is\n", m
mm = cholesky(m)
print "* factorization is\n", mm

* original matrix is
[[  4  12 -16]
 [ 12  37 -43]
 [-16 -43  98]]
* factorization is
[[ 2  6 -8]
 [ 6  1  5]
 [-8  5  3]]


We want to support reordering of vertices so that we can experiment with different ordering of vertices to find the best ordering to reduce fills:

In [2]:
# m: a matrix-representation of the pattern grpah
# f: a reordering function
def reorder(m, f):
    rows, cols = m.shape
    n = [[0]*cols for i in xrange(rows)]
    for i in xrange(rows):
        for j in xrange(cols):
            n[f(i)][f(j)] = m[(i, j)]
    return np.matrix(n)

This ```reorder``` function applies the reordering mapping function to m, and return the reordered matrix. And the ```reorder_func``` is the reordering mapping function

In [3]:
reorder_dict = {0:2, 1:0, 2:1}
def reorder_func(i):
    return reorder_dict[i]

Then, we can take a look at the cholesky factorization on the reordered matrix:

In [4]:
    m = np.matrix([[4, 12, -16], [12, 37, -43], [-16, -43, 98]])
    m_reordered = reorder(m, reorder_func)
    m_reordered_fac = cholesky(m_reordered)
    print "reordered cholesky is\n", m_reordered_fac


reordered cholesky is
[[ 6 -8  2]
 [-8  5  0]
 [ 2  0  0]]


Notice that this cholesky factorization is sparser than the previous one. Our goal is to determine which kind of reordering will lead to sparser cholesky factors.

In [5]:
def calc_sparsity(m):
    rows, cols = m.shape
    nonzeros = 0
    for i in xrange(rows):
        for j in xrange(cols):
            if (m[(i, j)] != 0):
                nonzeros += 1
    return float(nonzeros) / (rows * cols)
print "sparsity is %f, %f" %(calc_sparsity(mm), calc_sparsity(m_reordered_fac))

sparsity is 1.000000, 0.666667
